# Initial POI Selection

Since we are using a **Greedy Algorithm** for optimization, there is a high risk of falling into **local optima**.  
To prevent this, selecting **initial points** for locations where EVCS (Electric Vehicle Charging Stations) must be built is crucial in our research.

Our study stands out by categorizing the problem based on **Charger Type** and **Spatial Units** (Urban, Suburban, Rural). Each category is further divided into **cases**, and for each case, selecting the optimal initial point is essential for the effectiveness of our results.  
To address this, we apply different algorithms tailored to each case to ensure the best initial point selection.

Key steps include:
1. **Charger Type** categorization.
2. **Spatial Unit** division into urban, suburban, and rural areas.
3. Application of **case-specific algorithms** for initial point selection to avoid local optimization pitfalls.


## Case 1: Atlanta Level 2 EVCS Charger Initial Point Selection

The **Atlanta** area provides a diverse range of **POI candidates** with highly reliable **POI categories**.  
This allows us to identify potential locations for EVCS installations through effective **POI filtering**.

In this case, after conducting the **Initial Filtering** process, we further refine the results by using **GIS-based filtering** to increase precision. This additional step allows us to select the most suitable initial points for EVCS deployment.

### Process Overview:
1. **Initial POI Filtering** has already been completed, resulting in a list of potential EVCS installation sites.
2. **GIS-based Secondary Filtering** is applied to further refine the pre-filtered POIs, taking into account spatial factors such as accessibility, proximity to high-demand areas, and other geographic considerations.
3. Based on this enhanced filtering, the final **initial points** for EVCS installation are selected.


In [ ]:
import geopandas as gpd

gpkg_file_path = 'Path to Atlanta_POI_Candidate.gpkg'  # 실제 파일 경로로 수정하세요

# 2. GPKG 파일 읽기
poi_gdf = gpd.read_file(gpkg_file_path)

# 3. 필터링할 카테고리 리스트, GIS Tool 을 이용하여 Filltering
selected_category = ['bank','supermarket', 'department_store', 'hospital', 'public_building', 'mall', 'town_hall'] 
# 4. 'Category' 열에서 selected_category에 해당하는 행 필터링
filtered_poi_gdf = poi_gdf[poi_gdf['fclass'].isin(selected_category)]

# 결과 확인 (필터링된 데이터프레임 출력)
filtered_poi_gdf.head()

filtered_poi_gdf.to_file("Atlanta_init.gpkg", driver="GPKG")

## Case 2: Atlanta DCFC EVCS Charger Initial Point Selection

The **Atlanta** area is a key region for the installation of numerous **DCFC** (Direct Current Fast Chargers).  
Since the optimal location of DCFCs can be critical to traffic flow, we use the **road network** to guide the selection of initial points.

### Algorithm:

1. **Map demand values** using the **Motorway** and **Trunkway Demand Map**. Start by selecting POIs adjacent to the roads with the highest demand values based on the following criteria:
2. Find **POI candidates** near roads with high demand values, and ensure that the selected POIs maintain a **minimum distance of 3 km**, which is the minimum service radius for EVCS.
3. Select the specified number of **initial POIs**, repeating the process until the required number of points is reached.
4. If fewer than 5 POIs are selected in a specific area, the process reads the **Demand Map's pixel values** directly to identify high-demand regions and select additional POI candidates.


In [ ]:
# Import necessary libraries for geospatial data manipulation and processing
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.features import geometry_mask
import numpy as np
from shapely.geometry import box
from geopy.distance import geodesic

# Load geospatial datasets
motorway_gdf = gpd.read_file('Path to Your Road Network_Motorway.gpkg')  # Load motorway road network
trunk_gdf = gpd.read_file('Path to Your Road Network_Trunk.gpkg')  # Load trunk road network
poi_gdf = gpd.read_file('Path to your Atalanta_DCFC_Candidate.gpkg')  # Load candidate POIs for DCFC
geotif = rasterio.open('Path to your Demand Map.tif')  # Load demand map raster
boundary_gdf = gpd.read_file('Path to your Atlanta Polygon.gpkg')  # Load Atlanta boundary polygon

# Coordinate system transformation settings: Target CRS and WGS84 for distance calculations
target_crs = 'EPSG:3857'  # CRS for mapping
wgs84_crs = 'EPSG:4326'  # CRS for geodesic distance calculations (WGS 84)

# Transform the CRS of each dataset to the target CRS (EPSG:3857) if they don't match
if motorway_gdf.crs.to_string() != target_crs:
    motorway_gdf = motorway_gdf.to_crs(target_crs)
if trunk_gdf.crs.to_string() != target_crs:
    trunk_gdf = trunk_gdf.to_crs(target_crs)
if poi_gdf.crs.to_string() != target_crs:
    poi_gdf = poi_gdf.to_crs(target_crs)
if boundary_gdf.crs.to_string() != target_crs:
    boundary_gdf = boundary_gdf.to_crs(target_crs)

# Function to extract pixel values from the demand map for given geometries (roads)
def get_pixel_values(geometries, geotif):
    pixel_values = []
    for geom in geometries:
        # Create a mask from the geometry and extract pixel data from the demand map
        mask = geometry_mask([geom], transform=geotif.transform, invert=True, out_shape=(geotif.height, geotif.width))
        pixel_data = geotif.read(1, masked=True)[mask]  # Read the demand values where the geometry intersects
        if pixel_data.size > 0:
            pixel_values.append((geom, np.max(pixel_data)))  # Append the highest demand value for the geometry
        else:
            pixel_values.append((geom, None))  # Append None if no demand value is available
    return pixel_values

# Function to calculate the geodesic distance between two points in kilometers
def distance_in_km(point1, point2):
    return geodesic((point1.y, point1.x), (point2.y, point2.x)).km

# Function to check if a new POI is at least 3km away from existing selected POIs
def is_far_enough(selected_pois, new_poi_geom, min_distance_km=3):
    # Transform both selected POIs and new POI to WGS84 for distance calculation
    selected_pois_wgs84 = selected_pois.to_crs(wgs84_crs)
    new_poi_gdf = gpd.GeoDataFrame(geometry=[new_poi_geom], crs=selected_pois.crs)  # Ensure CRS consistency
    new_poi_wgs84 = new_poi_gdf.to_crs(wgs84_crs)

    # Check the geodesic distance between the new POI and each selected POI
    for poi in selected_pois_wgs84.itertuples():
        if distance_in_km(poi.geometry, new_poi_wgs84.geometry.iloc[0]) < min_distance_km:
            return False  # If the distance is less than the minimum distance (3km), return False
    return True  # If all distances are valid, return True

# Select 3 initial points for each region, ensuring a minimum distance of 3km between POIs
initial_candidates = []

for fid in boundary_gdf['fid'].unique():
    # Filter the boundary for each region (by fid)
    region_boundary = boundary_gdf[boundary_gdf['fid'] == fid]
    
    # Select motorway and trunk roads within the region boundary
    motorway_in_region = motorway_gdf[motorway_gdf.intersects(region_boundary.unary_union)]
    trunk_in_region = trunk_gdf[trunk_gdf.intersects(region_boundary.unary_union)]
    
    # Combine motorway and trunk roads into one GeoDataFrame
    combined_gdf = gpd.GeoDataFrame(pd.concat([motorway_in_region, trunk_in_region], ignore_index=True))
    
    # Get demand values for the combined road geometries
    combined_pixel_values = get_pixel_values(combined_gdf.geometry, geotif)
    combined_pixel_values = [v for v in combined_pixel_values if v[1] is not None]  # Filter out None values
    combined_pixel_values.sort(key=lambda x: x[1], reverse=True)  # Sort by demand value (descending)

    # Initialize an empty GeoDataFrame for storing selected POIs in the region
    region_candidates = gpd.GeoDataFrame(columns=poi_gdf.columns, crs=target_crs)

    # Loop through each road segment, selecting POIs based on proximity and demand
    for geom, _ in combined_pixel_values:
        candidates_in_pixel = poi_gdf[poi_gdf.intersects(geom)]  # Find POIs intersecting the road geometry
        for _, candidate_poi in candidates_in_pixel.iterrows():
            candidate_gdf = gpd.GeoDataFrame([candidate_poi], crs=poi_gdf.crs)  # Create GeoDataFrame for the candidate
            if len(region_candidates) > 0 and not is_far_enough(region_candidates, candidate_gdf.geometry.iloc[0]):
                continue  # Skip the POI if it's too close to already selected POIs
            region_candidates = pd.concat([region_candidates, candidate_gdf], ignore_index=True)  # Add the POI
            if len(region_candidates) >= 5:
                break  # Stop if 5 POIs have been selected for the region
        if len(region_candidates) >= 5:
            break  # Stop if 5 POIs have been selected for the region

    # If fewer than 5 POIs are selected, use demand map pixel values directly
    if len(region_candidates) < 5:
        pixel_data = geotif.read(1)  # Read demand map data
        sorted_indices = np.unravel_index(np.argsort(pixel_data, axis=None)[::-1], pixel_data.shape)  # Sort by demand
        
        # Loop through the highest demand pixels and find additional POIs
        for y, x in zip(sorted_indices[0], sorted_indices[1]):
            geom = box(*geotif.xy(y, x), *geotif.xy(y + 1, x + 1))  # Create a geometry for the pixel
            if region_boundary.unary_union.intersects(geom):  # Check if the pixel is within the region
                candidates_in_pixel = poi_gdf[poi_gdf.intersects(geom)]  # Find POIs in the pixel
                for _, candidate_poi in candidates_in_pixel.iterrows():
                    candidate_gdf = gpd.GeoDataFrame([candidate_poi], crs=poi_gdf.crs)  # Create GeoDataFrame for the POI
                    if len(region_candidates) > 0 and not is_far_enough(region_candidates, candidate_gdf.geometry.iloc[0]):
                        continue  # Skip the POI if it's too close to already selected POIs
                    region_candidates = pd.concat([region_candidates, candidate_gdf], ignore_index=True)
                    if len(region_candidates) >= 5:
                        break  # Stop if 5 POIs have been selected
            if len(region_candidates) >= 5:
                break  # Stop if 5 POIs have been selected

    initial_candidates.append(region_candidates)  # Add the selected candidates for the region

# Save the selected POIs to a GeoPackage file
if initial_candidates:
    initial_candidates_gdf = gpd.GeoDataFrame(pd.concat(initial_candidates, ignore_index=True), crs=target_crs)
    initial_candidates_gdf.to_file('Atlanta_DCFAST_init.gpkg', driver="GPKG")  # Save to GPKG
    print(f"Total selected POIs: {len(initial_candidates_gdf)}")  # Print the total number of selected POIs
else:
    print("No POIs were selected.")  # Print message if no POIs were selected

## Case 3: Suburban Level 2 EVCS Charger Initial Point Selection

In **Suburban areas**, POI candidates and their categories are diverse and highly reliable.  
Unlike other areas, the selection is influenced more by the **building category of the POI** than by the road network.

Therefore, a **weighted filtering** process is applied using the **Demand Map** and **POI building categories**. The algorithm proceeds as follows:

### Algorithm:

1. **Select priority POI categories**: This process is performed using GIS, where certain POI categories are prioritized for selection.
2. Assign **Demand Map values** to POI candidates, linking the demand levels to the POIs.
3. **Parse through the suburban polygon**, selecting POIs as initial points where demand is high and the POI matches a priority category.
4. If no POI exists for the selected category in a given region, the algorithm selects the POI with the highest demand from the **Demand Map** as the initial point.


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from shapely.geometry import box
import random

# Set priority POI categories
selected_category = ['department_store', 'town_hall', 'hospital', 'market_place', 
                     'mall', 'public_building']

# File paths
poi_path = "Path to your Suburban_POI_Candidate.gpkg'"
polygon_path = 'Path to your Suburban Polygon.gpkg'
demand_tif_path = 'Path to your Demand Map.tif'
output_path = "Path to your output path"

# 1. Load the target region polygons and POI data
polygon_gdf = gpd.read_file(polygon_path)  # Load suburban region polygons
poi_gdf = gpd.read_file(poi_path)  # Load POI candidate data

# 2. Read the demand data from the tif file and convert the pixels to polygons
with rasterio.open(demand_tif_path) as src:
    affine = src.transform
    demand_data = src.read(1)  # Read the first band of the raster
    rows, cols = np.where(demand_data > 0)  # Select pixels with demand values
    
    demand_polygons = []
    demand_values = []
    
    for row, col in zip(rows, cols):
        # Convert the pixel coordinates to geographic coordinates (bottom-left and top-right)
        x_min, y_min = affine * (col, row)
        x_max, y_max = affine * (col + 1, row + 1)
        demand_polygons.append(box(x_min, y_min, x_max, y_max))  # Convert the pixel to a polygon
        demand_values.append(demand_data[row, col])  # Store the demand value

# 3. Convert the demand pixels into a GeoDataFrame
demand_gdf = gpd.GeoDataFrame({'demand': demand_values, 'geometry': demand_polygons}, crs="EPSG:3857")

# 4. Perform a spatial join to map demand values to POIs (assign demand to POI candidates)
poi_with_demand = gpd.sjoin(poi_gdf, demand_gdf, how='left', op='within')

# Select the POI with the highest demand in each polygon
selected_pois = []
for index, polygon in polygon_gdf.iterrows():
    # Filter POIs that are within the current polygon (POIs with mapped demand values)
    contained_poi = poi_with_demand[poi_with_demand.within(polygon.geometry)]

    if contained_poi.empty:
        continue
    
    # 1) Filter POIs with the highest demand value
    max_demand_value = contained_poi['demand'].max()
    highest_demand_pois = contained_poi[contained_poi['demand'] == max_demand_value]
    
    if highest_demand_pois.empty:
        continue  # Skip if no POI has a demand value
    
    # 2) Search for POIs belonging to the preferred categories
    preferred_pois = highest_demand_pois[highest_demand_pois['fclass'].isin(selected_category)]
    
    if not preferred_pois.empty:
        # Randomly select a POI from the preferred category
        selected_poi = preferred_pois.sample()
    else:
        # If no POI belongs to the preferred category, randomly select from the highest demand POIs
        selected_poi = highest_demand_pois.sample()
    
    # Store the selected POI
    selected_pois.append(selected_poi)

# Save the selected POIs to a GeoDataFrame
if selected_pois:
    selected_pois_gdf = gpd.GeoDataFrame(pd.concat(selected_pois, ignore_index=True), crs='EPSG:3857')
    
    # Trim column names to 10 characters (to avoid issues with ESRI Shapefile column name limits)
    selected_pois_gdf.columns = [str(col)[:10] for col in selected_pois_gdf.columns]
    
    # Save the result as a .gpkg file
    selected_pois_gdf.to_file(f"{output_path}Suburban_init.gpkg", driver="GPKG")
    
else:
    print("No POIs were selected.")


## Case 4: Suburban DCFC EVCS Charger Initial Point Selection

In **Suburban areas**, high-speed chargers (DCFC) tend to be installed at intersections of major highways. To reflect this trend, the selection process considered the **road network**, **demand map**, and **POI** data. The DCFC EVCS locations in suburban areas are generally clearer compared to urban settings, and because the number of POI candidates is relatively low, running a full optimization was deemed unnecessary. Therefore, the following algorithm was used to install DCFC EVCS in all suburban areas.

### Algorithm:

1. **Selecting POIs near Motorway and Trunkway Intersections**:
   - Using **Motorway** and **Trunkway** road network data, the algorithm identifies intersections.
   - For each intersection, POIs within a **2410-meter radius** are searched.
   - Among the selected POIs, those belonging to **priority categories** (e.g., `department_store`, `hospital`, `market_place`, etc.) are prioritized.
   - If no POI is found within the priority categories, the nearest POI is randomly selected, and a **DCFC EVCS** is installed.

2. **Selecting POIs based on GeoTIFF Demand Data for Unselected Suburban Areas**:
   - For areas not covered by the intersection-based selection, the **GeoTIFF demand map** is used to ensure POIs are placed in areas with higher demand.
   - Using the `rasterio` library, the pixel value for each POI location is extracted from the demand map.
   - The algorithm first checks if a POI near a high-demand area belongs to a priority category, and if no such POI is found, it selects the nearest POI based on demand values to install the **DCFC EVCS**.

3. **Selecting POIs within Urban Polygons**:
   - The number of POIs to be selected in each **urban polygon** is determined by the variable **‘p’** associated with each polygon.
   - For each urban polygon, POIs are selected randomly, prioritizing those in **priority categories**. If the selected number of POIs is insufficient, additional POIs are selected based on **GeoTIFF demand data**.
   - If the polygon contains fewer POIs than required, additional POIs are chosen near high-demand pixels from the demand map, and the **DCFC EVCS** is installed.

By following the above steps, **DCFC EVCS** installations were successfully allocated across all suburban areas.


In [ ]:
import geopandas as gpd
import rasterio
import pandas as pd
import numpy as np
from shapely.geometry import Point
from shapely.ops import unary_union
from tqdm import tqdm

# File paths
urban_polygon_path = 'Path to your Suburban EVCS Polygon.gpkg'  # Urban Polygon
motorway_path = 'Path to your Motorways.gpkg'  # Motorway
trunk_path = 'Path to your  Trunk.gpkg'  # Trunk
poi_path = 'Path to your Suburban_DCFC_candidate.gpkg'  # POI
tif_file = 'Path to your Demand map'  # GeoTIFF

# Parameters
radius_meters = 3000  # 3 km 반경
selected_category = [
    'department_store', 'bank', 'hotel', 'theatre', 'town_hall', 'hospital', 
    'market_place', 'mall', 'public_building', 'cinema', 'vending_parking', 
    'general', 'cafe', 'fast_food', 'pharmacy', 'clothes'
]

# Load data
urban_polygon_gdf = gpd.read_file(urban_polygon_path)
motorway_gdf = gpd.read_file(motorway_path)
trunk_gdf = gpd.read_file(trunk_path)
poi_gdf = gpd.read_file(poi_path)

# Ensure correct CRS (EPSG:3857)
if urban_polygon_gdf.crs.to_string() != 'EPSG:3857':
    urban_polygon_gdf = urban_polygon_gdf.to_crs('EPSG:3857')

if motorway_gdf.crs.to_string() != 'EPSG:3857':
    motorway_gdf = motorway_gdf.to_crs('EPSG:3857')

if trunk_gdf.crs.to_string() != 'EPSG:3857':
    trunk_gdf = trunk_gdf.to_crs('EPSG:3857')

if poi_gdf.crs.to_string() != 'EPSG:3857':
    poi_gdf = poi_gdf.to_crs('EPSG:3857')

# 1. Motorway와 Trunk 간의 교차점 추출
motorway_union = unary_union(motorway_gdf.geometry)
trunk_union = unary_union(trunk_gdf.geometry)

intersection_points = motorway_union.intersection(trunk_union)

# 교차점이 여러 개일 경우 MultiPoint 처리
if intersection_points.geom_type == 'MultiPoint':
    intersection_points_gdf = gpd.GeoDataFrame(geometry=list(intersection_points.geoms), crs='EPSG:3857')
else:
    intersection_points_gdf = gpd.GeoDataFrame(geometry=[intersection_points], crs='EPSG:3857')

# 2. 교차점 반경 내에서 POI 찾기
def get_pois_within_radius(intersection_gdf, poi_gdf, radius_meters):
    selected_pois = []

    for point in intersection_gdf.geometry:
        # POI와 Intersection Point 사이의 거리 계산
        poi_gdf['distance'] = poi_gdf.geometry.apply(lambda x: point.distance(x))
        
        # 반경 내에 있는 POI 추출
        pois_within_radius = poi_gdf[poi_gdf['distance'] <= radius_meters]
        
        # 선택된 POI 추가
        selected_pois.append(pois_within_radius)
    
    # 모든 POI 합치기
    return gpd.GeoDataFrame(pd.concat(selected_pois, ignore_index=True), crs=poi_gdf.crs)

# 3. GeoTIFF에서 픽셀 값을 추출해 POI에 할당
def get_nearest_tif_value(poi_gdf, tif_file):
    with rasterio.open(tif_file) as tif:
        tif_data = tif.read(1)  # 첫 번째 밴드 읽기
        transform = tif.transform

        # POI의 좌표에 대응하는 GeoTIFF 픽셀 값 할당
        poi_gdf['tif_value'] = poi_gdf.geometry.apply(lambda point: get_pixel_value(point, tif_data, transform))

    return poi_gdf

# 특정 좌표에 대해 GeoTIFF에서 픽셀 값을 반환하는 함수
def get_pixel_value(point, tif_data, transform):
    row, col = ~transform * (point.x, point.y)  # 좌표를 픽셀 좌표로 변환
    row, col = int(row), int(col)

    if 0 <= row < tif_data.shape[0] and 0 <= col < tif_data.shape[1]:
        return tif_data[row, col]
    else:
        return np.nan  # 범위 밖일 경우 NaN 반환

# GeoTIFF 값이 높은 곳에 POI를 배치하는 함수
def assign_pois_from_tif(urban_polygon, tif_file, num_pois_needed, pois_gdf):
    with rasterio.open(tif_file) as tif:
        transform = tif.transform
        tif_data = tif.read(1)
        
        # Urban Polygon 경계 내에서 GeoTIFF 값이 높은 픽셀 선택
        bounds = urban_polygon.geometry.bounds
        x_min, y_min, x_max, y_max = bounds
        
        pixel_values = []
        for x in np.arange(x_min, x_max, tif.res[0]):
            for y in np.arange(y_min, y_max, tif.res[1]):
                point = gpd.points_from_xy([x], [y])[0]
                if urban_polygon.geometry.contains(point):
                    row, col = ~transform * (x, y)
                    row, col = int(row), int(col)
                    if 0 <= row < tif_data.shape[0] and 0 <= col < tif_data.shape[1]:
                        pixel_value = tif_data[row, col]
                        pixel_values.append((point, pixel_value))

        # GeoTIFF 값이 높은 순서대로 정렬
        pixel_values = sorted(pixel_values, key=lambda x: x[1], reverse=True)

        # 선택된 픽셀에 해당하는 POI들을 탐색
        selected_pois = []
        for point, pixel_value in pixel_values:
            pois_near_pixel = pois_gdf[pois_gdf.distance(point) < tif.res[0]]
            pois_in_category = pois_near_pixel[pois_near_pixel['category'].isin(selected_category)]
            
            if not pois_in_category.empty:
                selected_pois.append(pois_in_category.sample(n=1))  # 우선순위 POI가 있으면 선택
            elif not pois_near_pixel.empty:
                selected_pois.append(pois_near_pixel.sample(n=1))  # 우선순위 POI가 없으면 다른 POI 선택
            
            if len(selected_pois) >= num_pois_needed:
                break

        return gpd.GeoDataFrame(pd.concat(selected_pois, ignore_index=True), crs=urban_polygon.crs)

# 4. 도시 폴리곤 안에서 POI 선택
def select_random_pois(urban_gdf, pois_gdf, tif_file):
    selected_pois = []
    pois_gdf = get_nearest_tif_value(pois_gdf, tif_file)

    for fid, urban_polygon in tqdm(urban_gdf.iterrows(), total=len(urban_gdf), desc="Processing Urban Polygons"):
        pois_within_boundary = pois_gdf[pois_gdf.within(urban_polygon.geometry)]
        num_pois_to_select = int(urban_polygon['p'])

        if len(pois_within_boundary) >= num_pois_to_select:
            selected_pois.append(pois_within_boundary.sample(num_pois_to_select))
        else:
            remaining_pois_needed = num_pois_to_select - len(pois_within_boundary)
            tif_pois = assign_pois_from_tif(urban_polygon, tif_file, remaining_pois_needed, pois_gdf)
            selected_pois.append(tif_pois)

    return gpd.GeoDataFrame(pd.concat(selected_pois, ignore_index=True), crs=pois_gdf.crs)

# 교차점 반경 내에서 POI 선택
pois_near_intersections = get_pois_within_radius(intersection_points_gdf, poi_gdf, radius_meters)

# 도시 폴리곤 내에서 POI를 선택하고 GeoTIFF로 추가 배치
selected_pois = select_random_pois(urban_polygon_gdf, poi_gdf, tif_file)

# 결과 저장
selected_pois.to_file('Suburban_DCFC_Selection.gpkg', driver='GPKG')

print(f"Total selected POIs: {len(selected_pois)}")

## Case 5: Rural Level 2 EVCS Charger Initial Point Selection  
## Case 6: Rural DCFC EVCS Charger Initial Point Selection

For **Case 5** and **Case 6**, which pertain to **Rural areas**, the demand for fast charging (DCFC) is typically low. Most charging needs in rural areas are met with slower chargers (Level 2), and thus, the allocated **capacity** for EVCS in these regions is not high.

When installing **EVCS** and **DCFC** chargers in rural areas, a thorough **POI filtering process** was conducted, followed by the use of the **Demand Map** to determine the optimal locations for charger deployment. This ensured that chargers were placed where they would meet the actual demand of the rural population while making efficient use of available capacity.

### Algorithm Summary:

1. **Load Data**: Load rural polygons, POI candidate data, and demand map (GeoTIFF).

2. **Convert Demand Pixels to Polygons**: Extract demand values from the GeoTIFF file and convert them into polygons.

3. **Spatial Join**: Perform a spatial join between the POI data and demand polygons to assign demand values to POIs.

4. **POI Selection**:
   - Filter POIs within each rural polygon.
   - Select the POI with the highest demand, prioritizing those in **priority categories**.
   - If no priority category POI exists, randomly select the highest demand POI.

5. **DCFC & Level2 Type Are Same Algorithm in Here** :

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from shapely.geometry import box
import random

# Define priority POI categories
selected_category = ['department_store', 'town_hall', 'hospital', 'market_place', 
                     'mall', 'public_building']

# File paths
poi_path = 'Path to your Rural_DCFC_candidate.gpkg'  # POI# Path to POI candidate data
polygon_path = "Path to your Rural Polygon.gpkg"  # Path to rural polygons
demand_tif_path = 'Path to your Demand map'
output_path = "Path to your Output Path"  # Output directory

# 1. Load target region polygons and POI data
polygon_gdf = gpd.read_file(polygon_path)  # Load rural county polygons
poi_gdf = gpd.read_file(poi_path)  # Load POI candidate data

# 2. Read the demand data from the tif file and convert demand pixels to polygons
with rasterio.open(demand_tif_path) as src:
    affine = src.transform
    demand_data = src.read(1)  # Read the first band (demand values)
    rows, cols = np.where(demand_data > 0)  # Select only pixels with demand values
    
    demand_polygons = []
    demand_values = []
    
    for row, col in zip(rows, cols):
        # Convert pixel coordinates to geographic coordinates (bottom-left and top-right)
        x_min, y_min = affine * (col, row)
        x_max, y_max = affine * (col + 1, row + 1)
        demand_polygons.append(box(x_min, y_min, x_max, y_max))  # Convert the pixel to a polygon
        demand_values.append(demand_data[row, col])  # Store the demand value

# 3. Convert demand pixels into a GeoDataFrame
demand_gdf = gpd.GeoDataFrame({'demand': demand_values, 'geometry': demand_polygons}, crs="EPSG:3857")

# 4. Perform a spatial join to map demand values to POIs
poi_with_demand = gpd.sjoin(poi_gdf, demand_gdf, how='left', op='within')

# Select the POI with the highest demand in each polygon
selected_pois = []
for index, polygon in polygon_gdf.iterrows():
    # Filter POIs within the current polygon (POIs with mapped demand values)
    contained_poi = poi_with_demand[poi_with_demand.within(polygon.geometry)]

    if contained_poi.empty:
        continue  # Skip if no POIs are found in the polygon
    
    # 1) Filter POIs with the highest demand value
    max_demand_value = contained_poi['demand'].max()
    highest_demand_pois = contained_poi[contained_poi['demand'] == max_demand_value]
    
    if highest_demand_pois.empty:
        continue  # Skip if no POI has a demand value
    
    # 2) Search for POIs belonging to the preferred categories
    preferred_pois = highest_demand_pois[highest_demand_pois['fclass'].isin(selected_category)]
    
    if not preferred_pois.empty:
        # Randomly select a POI from the preferred category
        selected_poi = preferred_pois.sample()
    else:
        # If no POI belongs to the preferred category, randomly select from the highest demand POIs
        selected_poi = highest_demand_pois.sample()
    
    # Store the selected POI
    selected_pois.append(selected_poi)

# Save the selected POIs to a GeoDataFrame
if selected_pois:
    selected_pois_gdf = gpd.GeoDataFrame(pd.concat(selected_pois, ignore_index=True), crs='EPSG:3857')
    
    # Truncate column names to 10 characters (to comply with ESRI Shapefile column name limits)
    selected_pois_gdf.columns = [str(col)[:10] for col in selected_pois_gdf.columns]
    
    # Save the result as a .gpkg file
    selected_pois_gdf.to_file(f"{output_path}RC_DCFC_EVCS.gpkg", driver="GPKG")
    
else:
    print("No POIs were selected.")  # Print a message if no POIs were selected
